- Una sección llamada __Preliminares__ donde se realiza la descripción del problema y objetivos, así como explicar cómo implementarán su solución. Debe considerar qué criterios de optimización y métricas de desempeño.
- Una sección llamada __Aspectos computacionales__ donde se describirán las librerías y módulos a implementar, así como las funciones generadas y su objetivo.

# __Aspectos computacionales__

Seran añadidos a medida que sean requeridos los diversos import de librerias, no se generaran librerias externas, serin explicitas dentro de este notebook

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
import os
import preproc_nyc_sqf as preproc
import helpers

warnings.simplefilter('ignore')
plt.rcParams['figure.figsize'] = (15,12)
plt.style.use('seaborn')

In [20]:
df = pd.read_csv('2009_1perc.csv').drop(columns='Unnamed: 0')

In [21]:
df.head()

,year,pct,ser_num,datestop,timestop,recstat,inout,trhsloc,perobs,crimsusp,...,zip,addrpct,sector,beat,post,xcoord,ycoord,dettypcm,linecm,detailcm
0,2009,41,1779,4032009,130,A,O,P,1.0,CPW,...,,41,G,7,,1013067,0238633,CM,1,20
1,2009,108,5805,10292009,1050,A,O,P,3.0,BURG,...,,108,J,,,1012043,0212157,CM,1,14
2,2009,43,8340,10062009,1450,1,O,P,1.0,MISD,...,,43,E,,,1017599,0240200,CM,1,20
3,2009,77,932,1232009,1843,A,O,P,5.0,MIS,...,,77,J,4,35,1002625,0183442,CM,1,24
4,2009,110,11224,12132009,1655,A,O,P,3.0,CPW,...,,110,H,,,1024535,0209890,CM,1,20


In [22]:
proc_df, suitable_categorical_attributes, suitable_numerical_attributes = preproc.create_suitable_dataframe(df)

['perobs']


In [23]:
proc_df.head()

,recstat,inout,trhsloc,typeofid,explnstp,othpers,arstmade,sumissue,offunif,officrid,...,race,haircolr,eyecolor,build,city,sector,beat,post,month,meters
0,A,O,P,P,Y,N,N,N,Y,N,...,B,BK,BR,M,BRONX,G,7,,4,1.798320
2,1,O,P,V,Y,N,N,N,N,N,...,Q,BR,BR,M,BRONX,E,U,,10,1.920240
3,A,O,P,P,Y,N,N,N,Y,N,...,B,BK,BR,T,BROOKLYN,J,4,35,12,1.676400
5,1,I,T,V,Y,N,N,N,N,Y,...,B,BK,BR,M,MANHATTAN,F,16,,5,1.767840
6,1,O,P,P,Y,N,N,N,Y,N,...,B,BK,BR,T,BROOKLYN,N,U,,11,1.557528


Al utilizar la funcion auxiliar, nos permitio para igualar la cantidad de atributos en el benchmark, eliminar nulos, entre otros, lo cual nos permite tener un dataset ajustado para iniciar el preprocesamiento

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import factor_analyzer as factor
import missingno as msngo

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

ModuleNotFoundError: No module named 'factor_analyzer'

Dado este problema busca analizar la posibilidad de generar un arresto, se intenta resolver este problema con como un modelo de clasificacion, intentamos obtener una respuesta binaria de si o no.

In [25]:
proc_df['arstmade'].value_counts('%')

N    0.939387
Y    0.060613
Name: arstmade, dtype: float64

In [29]:
proc_df.columns.values

array(['recstat', 'inout', 'trhsloc', 'typeofid', 'explnstp', 'othpers',
       'arstmade', 'sumissue', 'offunif', 'officrid', 'frisked',
       'searched', 'contrabn', 'pistol', 'riflshot', 'asltweap',
       'knifcuti', 'othrweap', 'pf_hands', 'pf_wall', 'pf_grnd',
       'pf_drwep', 'pf_ptwep', 'pf_baton', 'pf_hcuff', 'pf_pepsp',
       'pf_other', 'radio', 'ac_rept', 'ac_inves', 'rf_vcrim', 'rf_othsw',
       'ac_proxm', 'rf_attir', 'cs_objcs', 'cs_descr', 'cs_casng',
       'cs_lkout', 'rf_vcact', 'cs_cloth', 'cs_drgtr', 'ac_evasv',
       'ac_assoc', 'cs_furtv', 'rf_rfcmp', 'ac_cgdir', 'rf_verbl',
       'cs_vcrim', 'cs_bulge', 'cs_other', 'ac_incid', 'ac_time',
       'rf_knowl', 'ac_stsnd', 'ac_other', 'sb_hdobj', 'sb_outln',
       'sb_admis', 'sb_other', 'rf_furt', 'rf_bulg', 'offverb', 'offshld',
       'sex', 'race', 'haircolr', 'eyecolor', 'build', 'city', 'sector',
       'beat', 'post', 'month', 'meters'], dtype=object)

Las metricas han sido declaras en sus import, se utilizaran curvas roc, classification report y confussion matrix, entre otros

Para procesar la data utilizaremos labelencoder como primera estrategia debido a su gran poder de transformar la data.

Para separar las muestras trabajaremos con el 20% de test y el 80 de entrenamiento

In [9]:
from sklearn.preprocessing import LabelEncoder

In [34]:
lbl = LabelEncoder()
labels = ['inout','trhsloc','typeofid','explnstp','othpers','arstmade','sumissue','offunif','officrid','frisked','searched','contrabn','pistol','riflshot','asltweap','knifcuti','othrweap','pf_hands','pf_wall','pf_grnd','pf_drwep','pf_ptwep','pf_baton','pf_hcuff','pf_pepsp','pf_other','radio','ac_rept','ac_inves','rf_vcrim','rf_othsw','ac_proxm','rf_attir','cs_objcs','cs_descr','cs_casng','cs_lkout','rf_vcact','cs_cloth','cs_drgtr','ac_evasv','ac_assoc','cs_furtv','rf_rfcmp','ac_cgdir','rf_verbl','cs_vcrim','cs_bulge','cs_other','ac_incid','ac_time','rf_knowl','ac_stsnd','ac_other','sb_hdobj','sb_outln','sb_admis','sb_other','rf_furt','rf_bulg','offverb','offshld','sex','race','haircolr','eyecolor','build']

In [35]:
for col in labels:
    proc_df[col] = lbl.fit_transform(proc_df[col])

In [45]:
X = proc_df.drop('arstmade',axis=1)
y = pd.DataFrame(proc_df.arstmade)

In [47]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=3367)

In [48]:
c_param_range= [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
gamma_param_range = [0.0000001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': c_param_range, 'gamma' : gamma_param_range}

In [49]:
gs_naives = GridSearchCV(MultinomialNB(),
                         {'alpha':np.linspace(0.1,1.0,10), 
                          'binarize': np.linspace(0.0,1.0,10), 
                          'fit_prior':[True,False], 
                          'class_prior':[None]},cv=5)
dec_tree_grid_cv = GridSearchCV(DecisionTreeClassifier(),
                               {'min_samples_split': np.linspace(0.1,1.0,10),
                               'criterion':['gini','entropy'],
                               'max_depth': np.linspace(1,32,32),
                               'min_samples_leaf': np.linspace(0.1,0.5,10),
                               },cv=5)
rf_cv = GridSearchCV(RandomForestClassifier(),{
                                    'n_estimators': [4, 6, 9], 
                                    'max_features': ['log2', 'sqrt','auto'], 
                                    'criterion': ['entropy', 'gini'],
                                    'max_depth': [2, 3, 5, 10], 
                                    'min_samples_split': [2, 3, 5],
                                    'min_samples_leaf': [1,5,8]
                                     },scoring = accuracy_score,cv=5)
ld_cv = GridSearchCV(LinearDiscriminantAnalysis(),{
                        'n_components':np.linspace(1,300,50), 
                        'priors':np.linspace(0.1,.9,20), 
                        'shrinkage':[None, 'auto'],
                        'solver':['svd','lsqr','eigen'], 
                        'store_covariance': [True,False], 
                        'tol':np.linspace(0.0001,1,50)
                        }, cv=5)
svc_cv = GridSearchCV(SVC(kernel='rbf'),param_grid,cv = 10)

Se adjuntan las grillas propuestas para los diversos modelos de evaluacion para el problema establecido, estas son para resolver problemas del tipo clasificacion